In [ ]:
import os
import json
import numpy as np
import cv2
import random
import pandas as pd
import matplotlib.pyplot as plt

# Đường dẫn đến thư mục dữ liệu
dataset_path = "/kaggle/input/btxrd-data/BTXRD/"
json_dir = os.path.join(dataset_path, "Annotations")
image_dir = os.path.join(dataset_path, "images")

# Lấy danh sách tất cả file JSON
json_files = [f for f in os.listdir(json_dir) if f.endswith(".json")]

# Chọn ngẫu nhiên 20 file JSON
random_json_files = random.sample(json_files, 20)
print(f"📊 Đã chọn {len(random_json_files)} ảnh ngẫu nhiên để phân tích.")

# Khởi tạo danh sách lưu thông tin
annotations_data = []
pixel_counts = []

# Duyệt qua từng file JSON đã chọn
for idx, json_file in enumerate(random_json_files, start=1):
    json_path = os.path.join(json_dir, json_file)
    
    # Đọc dữ liệu từ JSON
    with open(json_path, "r") as file:
        data = json.load(file)
    
    if "imagePath" in data and "shapes" in data:
        image_path = os.path.join(image_dir, data["imagePath"])
        
        # Kiểm tra kích thước ảnh
        if os.path.exists(image_path):
            image = cv2.imread(image_path)
            height, width, _ = image.shape
            total_pixels = height * width
            pixel_counts.append([f"Hình {idx}", total_pixels])
        else:
            height, width, total_pixels = 0, 0, 0
        
        # Duyệt qua từng vùng annotation
        tumor_areas = {}
        for shape in data["shapes"]:
            label = shape.get("label", "Unknown")
            points = np.array(shape.get("points", []), dtype=np.int32)
            
            # Tính diện tích của vùng annotation
            if len(points) > 2:
                area = cv2.contourArea(points)
            else:
                area = 0
            
            if label in tumor_areas:
                tumor_areas[label] += area
            else:
                tumor_areas[label] = area
        
        for label, area in tumor_areas.items():
            annotations_data.append([f"Hình {idx}", label, area])

        # Vẽ từng hình ảnh với vùng annotation
        if os.path.exists(image_path):
            img = cv2.imread(image_path)
            for shape in data["shapes"]:
                points = np.array(shape["points"], dtype=np.int32)
                cv2.polylines(img, [points], isClosed=True, color=(0, 255, 0), thickness=2)
            
            plt.figure(figsize=(6, 6))
            plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            plt.title(f"{data['imagePath']}")
            plt.axis("off")
            plt.show()

# Chuyển danh sách thành DataFrame
df = pd.DataFrame(annotations_data, columns=["Index", "Label", "Tumor Area"])
df_pixels = pd.DataFrame(pixel_counts, columns=["Index", "Total Pixels"])

# Loại bỏ dòng bị trùng (giữ lại dòng có diện tích lớn hơn)
df = df.sort_values(by=["Tumor Area"], ascending=False).drop_duplicates(subset=["Index", "Label"], keep="first")

# Sắp xếp theo thứ tự hình
df = df.sort_values(by=["Index"], key=lambda x: x.str.extract(r'(\d+)')[0].astype(int))
df_pixels = df_pixels.sort_values(by=["Index"], key=lambda x: x.str.extract(r'(\d+)')[0].astype(int))

print("📌 Thống kê theo thứ tự hình:")
print(df)
print("📌 Tổng số pixel của từng hình:")
print(df_pixels)

# Vẽ biểu đồ phân phối nhãn
plt.figure(figsize=(8, 4))
df["Label"].value_counts().plot(kind="bar", color="lightblue")
plt.title("Phân phối số lượng nhãn")
plt.xlabel("Loại tổn thương")
plt.ylabel("Số lượng")
plt.xticks(rotation=45)
plt.show()

# Vẽ histogram diện tích vùng annotation
plt.figure(figsize=(8, 4))
plt.hist(df["Tumor Area"], bins=20, color="salmon", edgecolor="black")
plt.title("Phân phối diện tích vùng annotation")
plt.xlabel("Diện tích (pixel)")
plt.ylabel("Số lượng")
plt.show()
